<a href="https://colab.research.google.com/github/deeprajbasu/MachineLearningCore/blob/main/sequential_data_rnn%2BLSTM%2Bgru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sequential data
# examples, audio, video, date wise temperatiure data, stock price data, language etc

# context, memory. 

In [ ]:
# Recurrent Neural Network 

# Hidden state, which remembers some information about a sequence

# current state : ht = f(ht-1,xt)
# ht -> current state
# ht-1 -> previous state
# xt -> input state


#output 
# yt = Why . ht
# Yt -> output
# Why -> weight at output layer
# ht -> current state



# An RNN remembers each and every piece of information through time. 
# ie. every sequence input influences state

# backpropagation is unstable as calculating derivatives is expensive


# context vs causation

In [2]:
# Long Short-Term Memory Model

In [ ]:
# maintaining context for Sequential data

# previous state

# rnn causes unstable gradient descent because ineffectient 'context' or 'memory'

#lstm:
# keep or forget data depending on the needs of the network.

# network can choose to forget data whenever information is not needed

#forget gate function kt

𝐾𝑡=𝜎(𝑊𝑘×[𝑆𝑡−1,𝑥𝑡]+𝐵𝑘)

𝑂𝑙𝑑𝑡=𝐾𝑡×𝑂𝑙𝑑𝑡−1

-----------------------

𝑥𝑡 current input features

𝑆𝑡−1 previous state

𝑊𝑘 forget gate weights

𝐵𝑘 forget gate bias

𝑂𝑙𝑑𝑡= previous state in network based on sigmoid output from forget gate

In [ ]:
# Gated Recurrent Unit

#reset gate - short term memery
#update gate  - long term memory 



In [96]:
import numpy as np
import tensorflow as tf

In [97]:
# Natural language processing with lstm 

# find out what is the next word

# sequential language context constructed by previous words.

# LSTM has capacity to maintain the model's state for over one thousand time steps

In [98]:
# word embedding

# systematic way to represent strings as vectors 

# 𝑉𝑒𝑐("𝐸𝑥𝑎𝑚𝑝𝑙𝑒")=[0.02,0.00,0.00,0.92,0.30,…]

In [99]:
# generate natural language with lstm

In [100]:

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')


print(f'Length of text: {len(text)} characters')


Length of text: 1115394 characters


In [101]:
text[0:200]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you'

In [102]:
# vocabulary from training data for embeddings
vocab = sorted(set(text))
vocab[0:10]

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3']

In [103]:
#vectorize string input

#reconstruct strings from vector for text generation 

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


example_texts = ['Citizen', 'resolved','e']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[16, 48, 59, 48, 65, 44, 53], [57, 44, 58, 54, 51, 61, 44, 43], [44]]>

In [104]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
print(text_from_ids(ids))

tf.Tensor([b'Citizen' b'resolved' b'e'], shape=(3,), dtype=string)


In [105]:
# Training Data

#id dataset from all of text
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


F
i
r
s
t
 
C
i
t
i


In [106]:

seq_length = 24
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

#print sequence of training data 
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())



b'First Citizen:\nBefore we '
b'proceed any further, hear'
b' me speak.\n\nAll:\nSpeak, s'
b'peak.\n\nFirst Citizen:\nYou'
b' are all resolved rather '


In [107]:
#model needs to predict next character
# input data and target data example
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

print(split_input_target(list("12345")))

dataset = sequences.map(split_input_target)

#X,y for each sequences
for input_example, target_example in dataset.take(3):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


(['1', '2', '3', '4'], ['2', '3', '4', '5'])
Input : b'First Citizen:\nBefore we'
Target: b'irst Citizen:\nBefore we '
Input : b'proceed any further, hea'
Target: b'roceed any further, hear'
Input : b' me speak.\n\nAll:\nSpeak, '
Target: b'me speak.\n\nAll:\nSpeak, s'


In [110]:
# Shuffle sequences
BATCH_SIZE = 100

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000

# dataset = (
#     dataset
#     .shuffle(BUFFER_SIZE)
#     .batch(BATCH_SIZE, drop_remainder=True)
#     .prefetch(tf.data.experimental.AUTOTUNE))

for input_example, target_example in dataset.take(2):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : [b'nce; our\nsufferance is a' b'eanness that\nafflicts us'
 b'be done: away, away!\n\nSe' b'feits on would relieve u'
 b': if they\nwould yield us' b'\n\nFirst Citizen:\nVery we'
 b"ore talking on't; let it" b'rt, but that he pays him'
 b'cond Citizen:\nNay, but s' b' good.\nWhat authority su'
 b' in hunger for bread, no' b"\nAll:\nWe know't, we know"
 b' me speak.\n\nAll:\nSpeak, ' b'articularise their abund'
 b'elf with being proud.\n\nS' b" kill him, and we'll hav"
 b'y he did it to\nplease hi' b'Second Citizen:\nWould yo'
 b'First Citizen:\nFirst, yo' b'l; and could be content '
 b'virtue.\n\nSecond Citizen:' b'ight guess they relieved'
 b', is as an\ninventory to ' b'very dog to the commonal'
 b'ge this with\nour pikes, ' b'en of accusations;\nhe ha'
 b'od citizens.\n\nFirst Citi' b'u, what he hath done fam'
 b'risen: why stay we prati' b' corn at our own price.\n'
 b'All:\nCome, come.\n\nFirst ' b"ther side o' the city\nis"
 b' the object of our miser' b'ond Citizen:\nOne

In [89]:
# build model


# embedding layer (input layer map character id to vector, trainable)
# lstm layer (rnn layer)

# dense layer (output layer, likeleyhood of each character in vocab)

In [158]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

num_steps = 20

In [159]:
#define the model 
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [160]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [161]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 24, 66) # (batch_size, sequence_length, vocab_size)


In [163]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

model.summary()


Model: "my_model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    multiple                  16896     
                                                                 
 gru_3 (GRU)                 multiple                  3938304   
                                                                 
 dense_11 (Dense)            multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [166]:
# Training the model 
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)


Epoch 1/20
697/697 [==============================] - 1005s 1s/step - loss: 2.0416
Epoch 2/20
  8/697 [..............................] - ETA: 21:01 - loss: 1.8674

KeyboardInterrupt: ignored

In [167]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states


In [168]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [170]:
import time
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


ROMEO:
First this disstowth cize him ither
Was common thy gildand, what eschize, this
Liss here for't, cationf ground.

SlAvION:
Comine,
He crosh a vingue
Turf they? that
is very all,
And, one awry
thou dose fulseloved-caunion say is no coussion that the eries 'tis yaur that withar in fhere?

SO:
Houts, but the vilit toll, readinty
I'll gentle brend, it woned, curizen,
'Tis thur,--

SOBASTIAN:
Your crising heart. 
MARINAN:
I talizes veny froth your dutlts. Of first forgunellict
of not is as that confeart, stors;
Be it betend let my art again: Vencusinits,
And hold thou sitner.



BAPNISABIIO:
Nrant age as,
For groul hastard never ging not; that for
Ay anl-my tuncys,
Sir, my madistresess the mage?

LO:
Shourth yeers looks, with rich falsely somes, thy bribon:
Mone the ispast.

CAN?

CANII:
WhO cross Crainy Well
sonst, wilt thene crecuized:
Which you confenter.

SEBASTIAN:
Which I make pidgivences fentensio have
selve that youl, and praifors untogain?

ASTANII:
In whiliraft them, this wh